In this notebook, the outputs generated for each scenario separately are combined for comparison of the scenarios. <br>
Plots are generated for visual comparison. Many different scenarios can be compared (i.e. more than 30).

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
output_path = '../output/'

In [ ]:
# must be adjusted, if additional scenarios are used (compare with list in 00_launcher)
scenarios = ['base', 'ref_35', 'policy_35', 'ass',
             'tax_7', 'tax_8', 'tax_9', 'tax_10', 'tax_11', 'tax_12',
             'prk_1', 'prk_2', 'prk_4', 'lim', 
             'pt_1', 'pt_2-4', 'pt_5', 'pt_6', 'pt_7', 'pt_9',
             'inf_1', 'inf_2', 'inf_3_run-rail', 'inf_4', 'inf_5', 'inf_6',
             'reg_1', 'reg_2', 'reg_3', 'reg_4', 'reg_5'
            ]

## Compare person kilometer shares by mode type between the scenarios

In [ ]:
# define output excel file to be read for every scenario (every folder in 'output')
file_name = 'results_total.xlsx'
sheet_name = 'pkm_shares_total'
unit = 'bn. pkm'
modes = ['air', 'bus', 'car', 'rail', 'walk', 'total']

# create a list with all paths to the files to be compared
file_list = []
for x in range(len(scenarios)):
    file_list.append(os.path.join(output_path, scenarios[x], file_name))

# read the data for every scenario and combine it in one list
excl_list = []
for x in range(len(file_list)):
    excl_list.append(pd.read_excel(file_list[x], sheet_name=sheet_name))
    
# convert the list to a pandas DataFrame for better handling
excl_merged = pd.DataFrame()
for excl_file in excl_list:
    excl_merged = excl_merged.append(excl_file, ignore_index=True)
excl_merged['mode']=excl_merged['Unnamed: 0']
excl_merged.drop(['Unnamed: 0'], axis=1, inplace=True)

# add the scenario name to the data in the pandas DataFrame
excl_merged['scenario'] = ""
for x in range(len(scenarios)):
    # add a row for 'air' if that row is missing (= no air travel)
    if excl_merged['mode'][x*6] != 'air':
        excl_merged.loc[x*6-0.5] = [0.0, 'air', np.nan]
        excl_merged = excl_merged.sort_index().reset_index(drop=True)
    # write the name in the scenario to all 
    for y in range(0, len(modes)):
        excl_merged['scenario'][y+x*6] = scenarios[x]
print(excl_merged.tail(40))
        
# define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
excl_merged = swap_columns(excl_merged, unit, 'scenario')
print(excl_merged)

In [ ]:
# plot the data, for every mode type their own plot with values for all scenarios in it
for mode in modes:
    pkm_mode = excl_merged.drop(excl_merged[excl_merged['mode'] != mode].index)
    # create bar chart
    fig = plt.figure(figsize=(10,5))
    plt.bar(pkm_mode['scenario'], pkm_mode[unit], width =0.7)
    plt.xticks(rotation=90)
    plt.xlabel('Scenario')
    plt.ylabel(f'{mode} {sheet_name} ({unit})')
    plt.savefig(output_path + '/scenario_comparison/' + mode + '_' + sheet_name + '_all_scen.pdf', 
                bbox_inches='tight')
    plt.close(fig)

## Compare (long-distance) emissions for different scenarios

In [ ]:
# still in work for combination of long-distance and short-distance emissions
# define output excel file to be read for every scenario (every folder in 'output')
file_name = 'results_total.xlsx'
sheet_name_1 = 'emissions_local'
sheet_name_2 = 'emissions_long'
modes_1 = ['bus', 'car', 'rail_short', 'walk']
modes_2 = ['air', 'bus', 'coach', 'rail_long', 'rail_short', 'car_long']
unit = 'mio tCO2eq'

# create a list with all paths to the files to be compared
file_list = []
for x in range(len(scenarios)):
    file_list.append(os.path.join(output_path, scenarios[x], file_name))

# read the data for every scenario and combine it in one list
excl_list = []
# local emissions
#for x in range(len(file_list)):
#    excl_list.append(pd.read_excel(file_list[x], sheet_name=sheet_name_1))
# long distance emissions
for x in range(len(file_list)):
    excl_list.append(pd.read_excel(file_list[x], sheet_name=sheet_name_2))
    
# convert the list to a pandas DataFrame for better handling
excl_merged = pd.DataFrame()
for excl_file in excl_list:
    excl_merged = excl_merged.append(excl_file, ignore_index=True)
#excl_merged['route_type'].fillna(excl_merged['Unnamed: 0'], inplace=True)
#excl_merged.rename({'route_type':'mode'}, axis=1, inplace=True)
#excl_merged.drop('Unnamed: 0', axis=1, inplace=True)
excl_merged.rename({'Unnamed: 0':'mode'}, axis=1, inplace=True)

# add the scenario name to the data in the pandas DataFrame
excl_merged['scenario'] = ""
# local emissions
#for x in range(len(scenarios)):
#    for y in range(len(modes_1)):
#        excl_merged['scenario'][y+x*len(modes_1)] = scenarios[x]
# long distance emissions
for x in range(len(scenarios)):
    for y in range(len(modes_2)):
        excl_merged['scenario'][y+x*len(modes_2)] = scenarios[x] #+len(modes_1)*len(scenarios)

        
# define function to swap columns
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
excl_merged = swap_columns(excl_merged, unit, 'scenario')
print(excl_merged.head(20))

In [ ]:
# plot the data, for every mode type their own plot with values for all scenarios in it
# only for emissions_long (mode_2)
for mode in modes_2:
    emiss = excl_merged.drop(excl_merged[excl_merged['mode'] != mode].index)
    # create bar chart
    fig = plt.figure(figsize=(10,5))
    plt.bar(emiss['scenario'], emiss[unit], width =0.7)  # xdata, ydata
    plt.xticks(rotation=90)
    plt.xlabel('Scenario')
    plt.ylabel(f'{mode} {sheet_name} ({unit})')
    plt.savefig(output_path + '/scenario_comparison/' + mode + '_' + sheet_name + '_all_scen.pdf', bbox_inches='tight')
    plt.close(fig)